In [1]:
import duckdb

In [2]:
con = duckdb.connect(':memory:')

In [6]:
df = con.execute("""SELECT DISTINCT conversation_hash, content FROM '../data/wildchat.parquet' JOIN '../data/nomic_filter_broad.csv' USING(conversation_hash) WHERE country = 'United States' AND role = 'user';""").fetch_df()

In [10]:
sample = df.sample(1000)

In [12]:
sample["classification"] = ""

In [13]:
sample.to_csv("../data/sample_for_annotation.csv", index=False)